In [1]:
import qiskit
import numpy as np
from qiskit import *
from math import pi

In [13]:
class CHSH_game:
    def __init__(self, rounds, eavesdrop=0): # eavesfrop = (0: none); (1: triplet entanglement); (2: intercept-resend)
        self.rounds = rounds
        self.eavesdrop = eavesdrop
        self.choices = [None, None]
#         self.basis = [[0, pi/8, pi/4], [0, pi/8, -pi/8], [0, 0, 0]]
        self.basis = [[0, pi/4, pi/2], [0, pi/4, -pi/4], [0, 0, 0]]
        self.game = [None]
        self.output = [[], []]
        self.key = [[], []]
        self.s_val = 0
        if eavesdrop == 1: 
            self.choices.append(None)
            self.output.append([])
            self.key.append([])
        
    def create_round(self):
        alice = QuantumRegister(1, 'alice')
        bob = QuantumRegister(1, 'bob')
        m_a = ClassicalRegister(1, 'm_a')
        m_b = ClassicalRegister(1, 'm_b')
        if self.eavesdrop == 0:
            qc = QuantumCircuit(alice, bob, m_a, m_b)
        elif self.eavesdrop == 1:
            eve = QuantumRegister(1, 'eve')
            m_e = ClassicalRegister(1, 'm_e')
            qc = QuantumCircuit(alice, bob, eve, m_a, m_b, m_e)
        return qc
    
    def create_epr(self, qc):
        epr = QuantumCircuit(2)
        epr.h(0)
        epr.cx(0, 1)
        qc.compose(epr, qubits=[0, 1], inplace=True)
        
    def create_eavesdropper(self, qc):
        entanglement = QuantumCircuit(2)
        entanglement.cx(0, 1)
        qc.compose(entanglement, qubits=[1, 2], inplace=True)
        
    def create_entanglement(self):
        pass
    
    def gen_random_choice(self, length):
        from os import urandom
        choice = [int.from_bytes(urandom(1), 'big') % 3 for _ in range(length)]
        return choice
    
    def measure(self, player): # player = 0 / 1: Alice / Bob
        if player != 2:
            choice = self.gen_random_choice(self.rounds)
        else:
            if self.eavesdrop == 1:
                choice = [0] * self.rounds # Eve should choose a strategy
        self.choices[player] = choice
        for i, c in enumerate(choice):
            basis = self.basis[player][c]
            self.game[i].ry(basis, player)
            self.game[i].measure(player, player)
            
    def retrieve_output(self):
        qasm_simulator = Aer.get_backend('qasm_simulator')
        for qc in self.game:
            result = execute(qc, qasm_simulator, shots=1, memory=True).result()
            memory = result.get_memory()[0].split()
            measured_bit_0 = int(memory[0])
            measured_bit_1 = int(memory[1])
            self.output[0].append(measured_bit_0)
            self.output[1].append(measured_bit_1)
            if self.eavesdrop == 1:
                measured_bit_2 = int(memory[2])
                self.output[2].append(measured_bit_2)
            
    def evaluate(self):
        # TODO: CHSH inequality
        test = [[], [], [], []] #01+02+21-22
        for i in range(self.rounds):
            if self.choices[0][i] == self.choices[1][i] and self.choices[0][i] != 2:
                self.key[0].append(self.output[0][i])
                self.key[1].append(self.output[1][i])
                if self.eavesdrop == 1:
                    self.key[2].append(self.output[2][i])
                continue
            ans = (-1) ** (self.output[0][i] + self.output[1][i])
            if self.choices[0][i] == 0 and self.choices[1][i] == 1:
                test[0].append(ans)
            elif self.choices[0][i] == 0 and self.choices[1][i] == 2:
                test[1].append(ans)
            elif self.choices[0][i] == 2 and self.choices[1][i] == 1:
                test[2].append(ans)
            elif self.choices[0][i] == 2 and self.choices[1][i] == 2:
                test[3].append(ans)
    
        avg = [0, 0, 0, 0]
        for i in range(4):
            item = test[i]
            if len(item) > 0:
                avg[i] = sum(item)/len(item)
        self.s_val = abs(sum(avg) - 2*avg[3])
#         print(avg)
        if self.s_val > 2:
            print ("Pass CHSH test! s value = %f" % self.s_val)
            diff = sum(list(a^b for a,b in zip(game.key[0],game.key[1])))
            print("Key length = %d" %len(game.key[0]))
            if diff > 0:
                print(" Key mismatach! diff: %d/%d" %(diff, len(game.key[0])))
        else:
            print ("Fail CHSH test! s value = %f" % self.s_val)
            diff_ab = sum(list(a^b for a,b in zip(game.key[0],game.key[1])))
            print("Key length = %d" %len(game.key[0]))
            if self.eavesdrop == 1:
                diff_ae = sum(list(a^b for a,b in zip(game.key[0],game.key[2])))
                diff_be = sum(list(a^b for a,b in zip(game.key[1],game.key[2])))
                print("Key mismatch: %d (a, b), %d (a, e), %d (b, e)" % (diff_ab, diff_ae, diff_be))
    
    def init(self):
        self.game = [self.create_round() for _ in range(self.rounds)]
        for qc in self.game:
            self.create_epr(qc)
            if self.eavesdrop == 1:
                self.create_eavesdropper(qc)
            qc.barrier()
            
    def start(self):
        self.init()
        self.measure(0) # Alice measure
        self.measure(1) # Bob measure
        if self.eavesdrop == 1: self.measure(2) # Eve measure
        self.retrieve_output()
        self.evaluate()

In [ ]:
rounds = 1000
game = CHSH_game(rounds, 1)
game.start()

In [ ]:
game.game[0].draw(output = 'mpl')